In [1]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 50)
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.feature_selection import RFE, RFECV
from sklearn.linear_model import LogisticRegression
import precision_recall_cutoff
import Feature_Importance_Funs

# reading updated train dataset
train = pd.read_csv('train_dataset.csv')
train = train.dropna()
train.shape

(1879, 41)

In [2]:
train.head()

trustLevel  totalScanTimeInSeconds  grandTotal  lineItemVoids  \
0           5                    1054       54.70              7   
1           3                     108       27.36              5   
2           3                    1516       62.16              3   
3           6                    1791       92.31              8   
4           5                     430       81.53              3   

   scansWithoutRegistration  quantityModifications  scannedLineItemsPerSecond  \
0                         0                      3                   0.027514   
1                         2                      4                   0.129630   
2                        10                      5                   0.008575   
3                         4                      4                   0.016192   
4                         7                      2                   0.062791   

   valuePerSecond  lineItemVoidsPerPosition  fraud  interaction_1  \
0        0.051898                  0.241379      0              0   
1        0.253333                  0.357143      0              0   
2        0.041003                  0.230769      0              0   
3        0.051541                  0.275862      0              0   
4        0.189605                  0.111111      0              0   

   interaction_2  interaction_3  interaction_4  average_seconds_per_item  \
0              0              0              1                     19.27   
1              0              0              1                      3.95   
2              0              0              1                     24.39   
3              0              0              1                     19.40   
4              0              0              1                      5.27   

   trustLevel_1  trustLevel_2  trustLevel_3  trustLevel_4  trustLevel_5  \
0             0             0             0             0             1   
1             0             0             1             0             0   
2             0             0             1             0             0   
3             0             0             0             0             0   
4             0             0             0             0             1   

   trustLevel_6  totalScanTimeInSeconds_0_1  grandTotal_0_1  \
0             0                    0.575178        0.547174   
1             0                    0.057955        0.273637   
2             0                    0.827775        0.621811   
3             1                    0.978130        0.923462   
4             0                    0.234008        0.815608   

   lineItemVoids_0_1  scansWithoutRegistration_0_1  quantityModifications_0_1  \
0           0.636364                           0.0                        0.6   
1           0.454545                           0.2                        0.8   
2           0.272727                           1.0                        1.0   
3           0.727273                           0.4                        0.8   
4           0.272727                           0.7                        0.4   

   scannedLineItemsPerSecond_0_1  valuePerSecond_0_1  \
0                       0.004045            0.001370   
1                       0.019364            0.006689   
2                       0.001204            0.001083   
3                       0.002347            0.001361   
4                       0.009337            0.005007   

   lineItemVoidsPerPosition_0_1  average_seconds_per_item_01  Labels  \
0                      0.021944                     0.000131       1   
1                      0.032468                     0.000027       3   
2                      0.020979                     0.000165       5   
3                      0.025078                     0.000132       2   
4                      0.010101                     0.000036       0   

   Labels_0  Labels_1  Labels_2  Labels_3  Labels_4  Labels_5  interaction_5  \
0         0         1         0         0         0         0       

In [3]:
# reading updated train dataset
test = pd.read_csv('test_dataset.csv')
test = test.dropna()
test.shape

(498121, 40)

In [4]:
test.head()

trustLevel  totalScanTimeInSeconds  grandTotal  lineItemVoids  \
0           4                     467       88.48              4   
1           3                    1004       58.99              7   
2           1                     162       14.00              4   
3           5                     532       84.79              9   
4           5                     890       42.16              4   

   scansWithoutRegistration  quantityModifications  scannedLineItemsPerSecond  \
0                         8                      4                   0.014989   
1                         6                      1                   0.026892   
2                         5                      4                   0.006173   
3                         3                      4                   0.026316   
4                         0                      0                   0.021348   

   valuePerSecond  lineItemVoidsPerPosition  interaction_1  interaction_2  \
0        0.189465                  0.571429              0              0   
1        0.058755                  0.259259              0              0   
2        0.086420                  4.000000              0              1   
3        0.159380                  0.642857              0              0   
4        0.047371                  0.210526              0              0   

   interaction_3  interaction_4  average_seconds_per_item  trustLevel_1  \
0              0              1                    5.2780             0   
1              0              1                   17.0198             0   
2              0              0                   11.5714             1   
3              0              1                    6.2743             0   
4              0              1                   21.1101             0   

   trustLevel_2  trustLevel_3  trustLevel_4  trustLevel_5  trustLevel_6  \
0             0             0             1             0             0   
1             0             1             0             0             0   
2             0             0             0             0             0   
3             0             0             0             1             0   
4             0             0             0             1             0   

   totalScanTimeInSeconds_0_1  grandTotal_0_1  lineItemVoids_0_1  \
0                    0.254645        0.884888           0.363636   
1                    0.548087        0.589959           0.636364   
2                    0.087978        0.140014           0.363636   
3                    0.290164        0.847985           0.818182   
4                    0.485792        0.421642           0.363636   

   scansWithoutRegistration_0_1  quantityModifications_0_1  \
0                           0.8                        0.8   
1                           0.6                        0.2   
2                           0.5                        0.8   
3                           0.3                        0.8   
4                           0.0                        0.0   

   scannedLineItemsPerSecond_0_1  valuePerSecond_0_1  \
0                       0.000481            0.001900   
1                       0.000878            0.000589   
2                       0.000188            0.000867   
3                       0.000859            0.001598   
4                       0.000693            0.000475   

   lineItemVoidsPerPosition_0_1  average_seconds_per_item_0_1  Labels  \
0                      0.051948                      0.000029       4   
1                      0.023569                      0.000094       3   
2                      0.363636                      0.000064       2   
3                      0.058442                      0.000034       0   
4                      0.019139                      0.000116       0   

   Labels_0  Labels_1  Labels_2  Labels_3  Labels_4  Labels_5  interaction_5  \
0         0         0         0         0         1         0       0.290909   
1         0         0   

In [5]:
## Engineering features using the strong heredity principle
train['heredity_1'] = train['interaction_1'] * train['trustLevel']

train['heredity_2'] = train['interaction_1'] * train['Labels']

train['heredity_3'] = train['trustLevel'] * train['Labels']

In [6]:
# Variable created in the last feature engineering section
train['interaction_9'] = np.where(train['heredity_1'] > 0.5, 1, 0)

In [ ]:
logit_list = list()
RF_list = list()
Ada_list = list()

for i in range(0,100):
    
    # Defining input and target variables
    X = train.drop(['fraud'], axis = 1)
    Y = train['fraud']

    # Splitting the data
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

    # Standardizing the dataset
    scaler = MinMaxScaler()
    X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
    X_test = pd.DataFrame(scaler.fit_transform(X_test), columns = X_test.columns)
    
    # Running RFE with LogisticRegression
    logit_rfe = RFE(estimator = LogisticRegression(), n_features_to_select = 5).fit(X_train, Y_train)# Extracting features that got 
    
    # Extracting features that got slected
    logit_list.append(X_train.columns[logit_rfe.support_])
    
    # Running RFE with random forest
    RF_rfe = RFE(estimator = RandomForestClassifier(n_estimators = 500, max_depth = 3), n_features_to_select = 5).fit(X_train, Y_train)

    # Extracting features that got slected
    RF_list.append(X_train.columns[RF_rfe.support_])
    
    # Running RFE with AdaBoost
    Ada_rfe = RFE(estimator = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500, learning_rate = 0.01), n_features_to_select = 5).fit(X_train, Y_train)

    # Extracting features that got slected
    Ada_list.append(X_train.columns[Ada_rfe.support_])

    if i == 0:
        print('Iteraction :', i, end=' ')
    else:
        print(i, end=' ')


Iteraction : 0 

In [ ]:
logit = pd.DataFrame(logit_list)
RF = pd.DataFrame(RF_list)
ada = pd.DataFrame(Ada_list)

logit.to_csv('logit_list.csv', index = False)
RF.to_csv('RF_list.csv', index = False)
ada.to_csv('ada_list.csv', index = False)

In [ ]:
final = pd.concat([logit, RF, ada], ignore_index = True)
final

In [13]:
logit = pd.read_csv('logit_list.csv')
logit.shape

(100, 5)

In [14]:
logit.head()

0              1                           2  \
0  totalScanTimeInSeconds  interaction_1               interaction_4   
1  totalScanTimeInSeconds  interaction_4  totalScanTimeInSeconds_0_1   
2  totalScanTimeInSeconds  interaction_4  totalScanTimeInSeconds_0_1   
3  totalScanTimeInSeconds  interaction_1               interaction_4   
4  totalScanTimeInSeconds  interaction_4  totalScanTimeInSeconds_0_1   

                            3              4  
0  totalScanTimeInSeconds_0_1  interaction_9  
1                  heredity_1  interaction_9  
2                  heredity_1  interaction_9  
3  totalScanTimeInSeconds_0_1  interaction_9  
4                  heredity_1  interaction_9

In [15]:
RF = pd.read_csv('RF_list.csv')
RF.shape

(100, 5)

In [16]:
RF.head()

0              1                           2  \
0  totalScanTimeInSeconds  interaction_1                  heredity_1   
1  totalScanTimeInSeconds  interaction_1  totalScanTimeInSeconds_0_1   
2  totalScanTimeInSeconds  interaction_1  totalScanTimeInSeconds_0_1   
3  totalScanTimeInSeconds  interaction_1  totalScanTimeInSeconds_0_1   
4  totalScanTimeInSeconds  interaction_1  totalScanTimeInSeconds_0_1   

            3              4  
0  heredity_2  interaction_9  
1  heredity_2  interaction_9  
2  heredity_1     heredity_2  
3  heredity_1  interaction_9  
4  heredity_1  interaction_9

In [17]:
ada = pd.read_csv('ada_list.csv')
ada.shape

(100, 5)

In [18]:
ada.head()

0                           1  \
0  scannedLineItemsPerSecond  totalScanTimeInSeconds_0_1   
1     totalScanTimeInSeconds   scannedLineItemsPerSecond   
2     totalScanTimeInSeconds    scansWithoutRegistration   
3              lineItemVoids  totalScanTimeInSeconds_0_1   
4     totalScanTimeInSeconds   scannedLineItemsPerSecond   

                               2                              3  \
0              lineItemVoids_0_1                  interaction_5   
1                  interaction_1                 grandTotal_0_1   
2                  interaction_1  scannedLineItemsPerSecond_0_1   
3  scannedLineItemsPerSecond_0_1                  interaction_6   
4  scannedLineItemsPerSecond_0_1    average_seconds_per_item_01   

                    4  
0          heredity_1  
1       interaction_6  
2  valuePerSecond_0_1  
3          heredity_1  
4       interaction_6

In [19]:
final = pd.concat([logit, RF, ada], ignore_index = True)
final

0                          1  \
0      totalScanTimeInSeconds              interaction_1   
1      totalScanTimeInSeconds              interaction_4   
2      totalScanTimeInSeconds              interaction_4   
3      totalScanTimeInSeconds              interaction_1   
4      totalScanTimeInSeconds              interaction_4   
..                        ...                        ...   
295    totalScanTimeInSeconds  scannedLineItemsPerSecond   
296    totalScanTimeInSeconds  scannedLineItemsPerSecond   
297  scansWithoutRegistration  scannedLineItemsPerSecond   
298    totalScanTimeInSeconds              lineItemVoids   
299    totalScanTimeInSeconds   scansWithoutRegistration   

                                2                             3  \
0                   interaction_4    totalScanTimeInSeconds_0_1   
1      totalScanTimeInSeconds_0_1                    heredity_1   
2      totalScanTimeInSeconds_0_1                    heredity_1   
3                   interaction_4    totalScanTimeInSeconds_0_1   
4      totalScanTimeInSeconds_0_1                    heredity_1   
..                            ...                           ...   
295                 interaction_1  lineItemVoidsPerPosition_0_1   
296             lineItemVoids_0_1  scansWithoutRegistration_0_1   
297                 interaction_1      average_seconds_per_item   
298  lineItemVoidsPerPosition_0_1                 interaction_5   
299     scannedLineItemsPerSecond                 interaction_1   

                              4  
0                 interaction_9  
1                 interaction_9  
2                 interaction_9  
3                 interaction_9  
4                 interaction_9  
..                          ...  
295               interaction_5  
296               interaction_9  
297  totalScanTimeInSeconds_0_1  
298                  heredity_1  
299  totalScanTimeInSeconds_0_1  

[300 rows x 5 columns]

In [29]:
final['0'].value_counts().head(1)

totalScanTimeInSeconds    254
Name: 0, dtype: int64

In [28]:
final['1'].value_counts().head(1)

interaction_1    150
Name: 1, dtype: int64

In [30]:
final['2'].value_counts().head(2)

totalScanTimeInSeconds_0_1    160
interaction_4                  50
Name: 2, dtype: int64

In [31]:
final['3'].value_counts().head(1)

heredity_1    125
Name: 3, dtype: int64

In [32]:
final['4'].value_counts().head(1)

interaction_9    206
Name: 4, dtype: int64

In [38]:
ranked_features = pd.DataFrame({"Top_5_Features": ['totalScanTimeInSeconds', 
                                                   'interaction_1', 
                                                   'interaction_4', 
                                                   'heredity_1',
                                                   'interaction_9']}, index = [1, 2, 3, 4, 5])
ranked_features.head()


Top_5_Features
1  totalScanTimeInSeconds
2           interaction_1
3           interaction_4
4              heredity_1
5           interaction_9